In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import numpy as np
import copy
import networkx as nx
from itertools import permutations,product,chain
import pydot

In [0]:

def recursiveparens_parser(s):
    starts = []
    count = 0
    level = 0
    parens = [[-1, s]]
    for i in range(len(s)):
        if s[i] == '(':
            if len(starts) == level:
                starts += [i]
            else:
                starts[level] = i
            level += 1
        elif s[i] == ')':
            level -= 1
            
            parens += [[level, s[starts[level]+1:i]]]
            count += 1
    if level > 0:
        return "Mismatched Parentheses"
    return parens

In [0]:
def parse(exp):
    stack = {}
    subexps = recursiveparens_parser(exp)
    maxlevel = max(subexps, key= lambda x: x[0])[0]
    totalexp = ''
    for i in range(maxlevel,-2,-1):
        count = 0
        for j in range(len(subexps)):
            for k in stack.keys():
                if ('(' + k + ')') in subexps[j][1]:
                    subexps[j][1] = subexps[j][1].replace(('(' + k + ')'),stack[k])
            if i >= 0 and subexps[j][0] == i and (subexps[j][1] not in stack.keys()):
                stack[subexps[j][1]] = 's' + str(i) + '_' + str(count)
                count += 1
    totalexp = min(subexps, key= lambda x: x[0])[1][1:-1]
    return totalexp,subexps,stack

In [0]:
def breakexp(s):
  a = list(map(lambda x: x.strip(),s.split('+')))
  b = list(map(lambda x: ','.join(list(map(lambda y: y.strip(),x.split('*')))), a))
  return b

  

In [0]:
def resolve(z):
  k = z.keys()
  it = z.values()
  zr = {str(y):x for x,y in z.items()}
  zn = {x:[] for x,y in z.items()}
  for ah in it:
    for n in ah:
          n1 = n.split(',')
          labels = []
          if any(n2 in k for n2 in n1):
            # iedges = G.in_edges(n)
            # oedges = G.out_edges(n)
            newedges = []
            indices = [i for i in range(len(n1)) if n1[i] in z.keys()]
            
            sub = []
            
            for i in indices:
              sub += [n1[i]]
            # print(sub)
            A = [z[a] for a in sub]
            PERMS = [list(p) for p in product(*A)]
            # print(PERMS)
            
            for geh in PERMS:
              subcopy = copy.deepcopy(n1)
              # print(geh)
              for p,i,j in zip(geh,indices,range(len(indices))):
                # print(i,j)
                subcopy[i] = p
              labels += [','.join(subcopy)]
            if len(labels) > 0:
              zn[zr[str(ah)]] += labels
                # print(subcopy)
              # print(labels)
          else:
            zn[zr[str(ah)]] = ah
  return zn
  

In [0]:
def alledges(a,b):
  i = b.find(a)
  if i == -1:
    return []
  elif i == 0:
    if b[len(a):][0] == ',':
      return [(a,b),(b[len(a)+1:],b)]
    elif b[len(a):][-1] == ',':
      return [(a,b),(b[len(a):-1],b)]
    else:
      return [(a,b),(b[len(a):],b)]
    
  else:
    if b[:i][-1] == ',':
      return [(a,b),(b[:i-1],b)]
    elif b[:i][0] == ',':
      return [(a,b),(b[1:i],b)]
    else:
      return [(a,b),(b[:i],b)]
    

In [0]:
def create_protocol(subexp,s,P=nx.DiGraph()):
    G=copy.deepcopy(P)
    d = {y:[x] for x,y in s.items()}
    z = {y:[] for x,y in s.items()}
    c = copy.deepcopy(subexp)
    # print(s)
    while len(c) > 0:
      m = max(c,key=lambda x: x[0])[0]
      k = (np.array(list(filter(lambda x: True if x[0]==m else False, c)))[:,1]).tolist()
      k0 = list(filter((lambda x: not any(a in x for a in d)),k))
      k1 = list(filter((lambda x: any(a in x for a in d)),k))
      # print(k0)
      for e in k:
        e = (e.replace('[','')).replace(']','')
        e1 = list(map(lambda x: x.strip(),e.split('+')))
        # print(e1)
        for e2 in e1:
          ei = e2.split('*')

          ei = ','.join(list(map(lambda x: x.strip(), ei)))
          if ei not in list(G.nodes()):
            G.add_node(ei)
        
        p = list(permutations(e1))
        

        for p1 in p:
          p2 = ','.join(list(map(lambda x: ','.join(list(map(lambda y: y.strip(),x.split('*')))),list(p1))))
          if e in s.keys():
            z[s[e]] += [p2]
          if p2 not in list(G.nodes()):
            G.add_node(p2)
            G.add_edges_from([(a if '*' not in a else (','.join((''.join(a.split())).split('*'))),p2) for a in e1])
        e3 = list(map(lambda x: list(map(lambda y: y.strip(),x.split('*'))) ,e1))
        for e4 in e3:
          # print(e4)
          for e5 in e4:
            if e5 not in list(G.nodes()):
              G.add_node(e5)
          # print([(a,','.join(e4)) for a in e4 if a != ','.join(e4)])
          G.add_edges_from([(a,','.join(e4)) for a in e4 if a != ','.join(e4)])
      z = resolve(z)     
      for n in list(G.nodes()):

        n1 = n.split(',')
        # print(n1)
        if any(n2 in z.keys() for n2 in n1):
          iedges = G.in_edges(n)
          oedges = G.out_edges(n)
          newedges = []
          indices = [i for i in range(len(n1)) if n1[i] in z.keys()]
          
          sub = []
          
          for i in indices:
            sub += [n1[i]]
          # print(sub)
          A = [z[a] for a in sub]
          PERMS = [list(p) for p in product(*A)]
          # print(PERMS)
          inlabels = []
          labels = []
          for geh in PERMS:
            subcopy = copy.deepcopy(n1)
            # print(geh)
            for p,i,j in zip(geh,indices,range(len(indices))):
              # print(i,j)
              subcopy[i] = p
            inlabels += [subcopy]
            labels += [','.join(subcopy)]
            # print(subcopy)
          # print(labels)
          # print(iedges)
          for v in iedges:
            dest = list(v)[0]
            lm = dest.split(',')
            # print(lm)
            dests = []
            indices = [i for i in range(len(lm)) if lm[i] in z.keys()]
            if len(indices) < 1:
              dests = [dest]
            else:
              for i in range(len(lm)):
                if i in indices:
                  dests += [z[lm[i]]]
                else:
                  dests += [[lm[i]]]
              dests = [','.join(list(p)) for p in product(*dests)]
            # print(dest,dests)
            newedges += [(d,a) for a,p in list(zip(labels,PERMS)) for d in dests if d in p]
          for v in oedges:
            dest = list(v)[1]
            if G.out_degree(dest) < 1:
              v1 = dest.split(',')
              # print(v)
              indices = [i for i in range(len(v1)) if v1[i] in z.keys()]          
              sub = []
              
              for i in indices:
                sub += [v1[i]]
              # print(sub)
              A = [z[a] for a in sub]
              PERMS = [list(p) for p in product(*A)]
              # print(PERMS)
              inlabels = []
              olabels = []
              for geh in PERMS:
                subcopy = copy.deepcopy(v1)
                # print(geh)
                for p,i,j in zip(geh,indices,range(len(indices))):
                  # print(i,j)
                  subcopy[i] = p
                inlabels += [subcopy]
                olabels += [','.join(subcopy)]
              dlabels = []
              for d1 in inlabels: 
                dlabels += list(chain.from_iterable([[[','.join(d1[:i]),','.join(d1[i:])]] for i in range(1,len(d1))]))
              # print(dlabels)
              newedges += list(chain.from_iterable([alledges(a,d2) for a in labels for d1,d2 in list(zip(inlabels,olabels)) if a in chain.from_iterable(dlabels)]))


          G.remove_node(n)
          G.add_edges_from(newedges)
  
      # print(k)
      for m in k:
        # print(m)
        for f in c:
          if f[1]==m:
            c.remove(f)
            break


      
    return G,z
    



In [0]:
def pairwiseGibson(part1,part2):
  protocol = ("For Gibson Isothermal Assembly:\nObtain 2.5 uL of 0.3 pmol "+part1+" \nObtain 2.5 uL of 0.3 pmol "+part2+"\n")
  generalProtocol = ("Create Gibson master mix by combining the following:\n\t40 uL ISO buffer\n\t1.6 uL T5 Exonuclease\n\t2.5 uL Phusion high-fidelity DNA polymerase\n\t20uL Taq ligase\n\t85.9 uL Water\nMix 10 uL of Gibson Master mix with the DNA samples and 5 uL of deionized water\nIncubate sample in a thermocycler at 50\u00b0C for 15 minutes\nStore sample at -20\u00b0C to await transformation into E. Coli\n")
  return ''.join([protocol, generalProtocol])

In [0]:
def recursiveprotocol(G,n):

  succ = list(G.successors(n))
  if len(succ) < 2:
    return ""
  A = copy.deepcopy(G)
  A = nx.bfs_tree(A,succ[0])
  B = copy.deepcopy(G)
  B = nx.bfs_tree(B,succ[1])
  return '\n'.join([recursiveprotocol(G,succ[0]),recursiveprotocol(G,succ[1]),pairwiseGibson(succ[0],succ[1])])

  

In [0]:
exp = input("Enter your expression : ")

Enter your expression : (a+b)*(c+d)


In [0]:

e,se,s = parse(exp)
G, z = create_protocol(se,s)
relabels = {}
for n in list(G.nodes):
  relabels[n] = "\"" + n + "\""
G = nx.relabel_nodes(G,relabels)
nx.nx_pydot.write_dot(G, "graph.dot")
P = G.reverse()
trees = [[n,nx.bfs_tree(P,n)] for n in P.nodes if P.in_degree(n)==0]

print(recursiveprotocol(trees[0][1],trees[0][0]))

    
  



For Gibson Isothermal Assembly:
Obtain 2.5 uL of 0.3 pmol "a" 
Obtain 2.5 uL of 0.3 pmol "b"
Create Gibson master mix by combining the following:
	40 uL ISO buffer
	1.6 uL T5 Exonuclease
	2.5 uL Phusion high-fidelity DNA polymerase
	20uL Taq ligase
	85.9 uL Water
Mix 10 uL of Gibson Master mix with the DNA samples and 5 uL of deionized water
Incubate sample in a thermocycler at 50°C for 15 minutes
Store sample at -20°C to await transformation into E. Coli



For Gibson Isothermal Assembly:
Obtain 2.5 uL of 0.3 pmol "c" 
Obtain 2.5 uL of 0.3 pmol "d"
Create Gibson master mix by combining the following:
	40 uL ISO buffer
	1.6 uL T5 Exonuclease
	2.5 uL Phusion high-fidelity DNA polymerase
	20uL Taq ligase
	85.9 uL Water
Mix 10 uL of Gibson Master mix with the DNA samples and 5 uL of deionized water
Incubate sample in a thermocycler at 50°C for 15 minutes
Store sample at -20°C to await transformation into E. Coli

For Gibson Isothermal Assembly:
Obtain 2.5 uL of 0.3 pmol "a,b" 
Obtain 2.